In [81]:
import pandas as pd
import numpy as np
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [82]:
#load the songs table and find the sessions
df_songs = pd.read_excel('Copy of Karaoke Dataset.xlsx', sheet_name='Karaoke Choices')
# df_songs['time elapsed'] = df_songs['Date'].diff(1)
df_songs['session'] = df_songs['Date'].diff(1).dt.total_seconds().ge(59*60).cumsum() + 1
df_songs

,Date,Artist,Song,session
0,2020-12-22 13:59:59.971,Wham!,Last Christmas,1
1,2020-12-22 15:00:00.000,Dolly Parton,9 To 5,2
2,2020-12-22 15:02:00.010,Camilla Cabello Ft. Young Thug,Havana,2
3,2020-12-22 15:04:00.019,Moana,How Far I’ll Go,2
4,2020-12-22 18:00:00.000,Backstreet Boys,I Want It That Way,3
...,...,...,...,...
983,2021-02-01 22:09:00.029,Kings Of Leon,Sex Is On Fire,298
984,2021-02-01 22:11:00.038,Hugh Jackman & The Greatest Showman Cast,The Greatest Show,298
985,2021-02-01 22:14:00.010,Lil Nas X,Old Town Road,298
986,2021-02-01 22:16:59.981,Ike And Tina Turner,Proud Mary,298


In [83]:
#load customers table
df_customers = pd.read_excel('Copy of Karaoke Dataset.xlsx', sheet_name='Customers')
df_customers['end time'] =  df_customers['Entry Time'] + pd.DateOffset(minutes=10)
df_customers = df_customers.rename(columns={'Entry Time':'Start', 'end time':'End'})

In [84]:
#prepare for cross join
df_songs['dummy'] = 1
df_customers['dummy'] = 1

In [85]:
#cross join and filter out those songs that dont take place within 10 minutes of the sessions start
df_pairings = pd.merge(df_songs, df_customers, on='dummy', how='outer')
df_pairings = df_pairings[df_pairings['Date'].between(df_pairings['Start'], df_pairings['End'])]
df_pairings

,Date,Artist,Song,session,dummy,Customer ID,Start,End
203,2020-12-22 13:59:59.971,Wham!,Last Christmas,1,1,cd2834,2020-12-22 13:54:00,2020-12-22 14:04:00
368,2020-12-22 15:00:00.000,Dolly Parton,9 To 5,2,1,2de3d7,2020-12-22 14:55:00,2020-12-22 15:05:00
669,2020-12-22 15:02:00.010,Camilla Cabello Ft. Young Thug,Havana,2,1,2de3d7,2020-12-22 14:55:00,2020-12-22 15:05:00
970,2020-12-22 15:04:00.019,Moana,How Far I’ll Go,2,1,2de3d7,2020-12-22 14:55:00,2020-12-22 15:05:00
1296,2020-12-22 18:00:00.000,Backstreet Boys,I Want It That Way,3,1,6990000000000000162183243612064360401218956014...,2020-12-22 17:51:00,2020-12-22 18:01:00
...,...,...,...,...,...,...,...,...
292978,2021-02-01 15:00:00.000,Journey,Don’t Stop Believin’,295,1,a0a25c,2021-02-01 14:51:00,2021-02-01 15:01:00
293731,2021-02-01 16:59:59.971,Hugh Jackman & The Greatest Showman Cast,The Greatest Show,296,1,80244a,2021-02-01 16:51:00,2021-02-01 17:01:00
294865,2021-02-01 21:00:00.000,The Lion King,I Just Can’t Wait To Be King,297,1,707b62,2021-02-01 20:55:00,2021-02-01 21:05:00
295011,2021-02-01 22:00:00.029,Estelle Ft. Kanye West,American Boy,298,1,cdda70,2021-02-01 21:53:00,2021-02-01 22:03:00


In [86]:
# join back to the songs table
df_output = pd.merge(df_pairings, df_songs, how='right', on=['Date', 'Artist', 'Song'])
df_output

,Date,Artist,Song,session_x,dummy_x,Customer ID,Start,End,session_y,dummy_y
0,2020-12-22 13:59:59.971,Wham!,Last Christmas,1.0,1.0,cd2834,2020-12-22 13:54:00,2020-12-22 14:04:00,1,1
1,2020-12-22 15:00:00.000,Dolly Parton,9 To 5,2.0,1.0,2de3d7,2020-12-22 14:55:00,2020-12-22 15:05:00,2,1
2,2020-12-22 15:02:00.010,Camilla Cabello Ft. Young Thug,Havana,2.0,1.0,2de3d7,2020-12-22 14:55:00,2020-12-22 15:05:00,2,1
3,2020-12-22 15:04:00.019,Moana,How Far I’ll Go,2.0,1.0,2de3d7,2020-12-22 14:55:00,2020-12-22 15:05:00,2,1
4,2020-12-22 18:00:00.000,Backstreet Boys,I Want It That Way,3.0,1.0,6990000000000000162183243612064360401218956014...,2020-12-22 17:51:00,2020-12-22 18:01:00,3,1
...,...,...,...,...,...,...,...,...,...,...
983,2021-02-01 22:09:00.029,Kings Of Leon,Sex Is On Fire,NaN,NaN,NaN,NaT,NaT,298,1
984,2021-02-01 22:11:00.038,Hugh Jackman & The Greatest Showman Cast,The Greatest Show,NaN,NaN,NaN,NaT,NaT,298,1
985,2021-02-01 22:14:00.010,Lil Nas X,Old Town Road,NaN,NaN,NaN,NaT,NaT,298,1
986,2021-02-01 22:16:59.981,Ike And Tina Turner,Proud Mary,NaN,NaN,NaN,NaT,NaT,298,1
